In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import copy 

import xml.etree.ElementTree as ET

import numpy as np

from astropy.table import Table

from urllib.parse import urlencode
from urllib.request import urlopen
from urllib.error import HTTPError

from bs4 import BeautifulSoup

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Sandbox exploring the arXiv advanced search function

#### Simple Search with 16 results

- [Search URL](https://arxiv.org/search/advanced?advanced=1&terms-0-operator=AND&terms-0-term=Galaxies&terms-0-field=title&terms-1-operator=AND&terms-1-term=Assembly&terms-1-field=abstract&classification-physics=y&classification-physics_archives=astro-ph&classification-include_cross_list=exclude&date-year=&date-filter_by=date_range&date-from_date=2020-12-01&date-to_date=2021-02-03&date-date_type=announced_date_first&abstracts=show&size=200&order=-announced_date_first)

In [4]:
# First simple search with only 16 results
search_1 = "https://arxiv.org/search/advanced?advanced=1&terms-0-operator=AND&terms-0-term=Galaxies&terms-0-field=title&terms-1-operator=AND&terms-1-term=Assembly&terms-1-field=abstract&classification-physics=y&classification-physics_archives=astro-ph&classification-include_cross_list=exclude&date-year=&date-filter_by=date_range&date-from_date=2020-12-01&date-to_date=2021-02-03&date-date_type=announced_date_first&abstracts=show&size=200&order=-announced_date_first"

In [5]:
response = urlopen(search_1)
result = response.read()

soup = BeautifulSoup(result, "html.parser")

lists = soup.find_all('li', class_=["arxiv-result"])

len(lists)

16

In [7]:
a = lists[0]

In [40]:
a

<li class="arxiv-result">
<div class="is-marginless">
<p class="list-title is-inline-block"><a href="https://arxiv.org/abs/2101.12634">arXiv:2101.12634</a>
<span> [<a href="https://arxiv.org/pdf/2101.12634">pdf</a>, <a href="https://arxiv.org/format/2101.12634">other</a>] </span>
</p>
<div class="tags is-inline-block">
<span class="tag is-small is-link tooltip is-tooltip-top" data-tooltip="Cosmology and Nongalactic Astrophysics">astro-ph.CO</span>
</div>
</div>
<p class="title is-5 mathjax">
      
        Morphology of relaxed and merging <span class="search-hit mathjax">galaxy</span> clusters. Analytical models for monolithic Minkowski functionals
      
    </p>
<p class="authors">
<span class="has-text-black-bis has-text-weight-semibold">Authors:</span>
<a href="/search/?searchtype=author&amp;query=Schimd%2C+C">C. Schimd</a>, 
      
      <a href="/search/?searchtype=author&amp;query=Sereno%2C+M">M. Sereno</a>
</p>
<p class="abstract mathjax">
<span class="search-hit">Abstract</sp

In [47]:
a.find_all('p')[1].text.replace('\n', '').strip()

'Morphology of relaxed and merging galaxy clusters. Analytical models for monolithic Minkowski functionals'

In [50]:
a.find_all('p', _class='title is-5 mathjax')

[]

In [53]:
b = a.find_all('p')[1]

In [55]:
b.

In [17]:
a

<li class="arxiv-result">
<div class="is-marginless">
<p class="list-title is-inline-block"><a href="https://arxiv.org/abs/2101.12634">arXiv:2101.12634</a>
<span> [<a href="https://arxiv.org/pdf/2101.12634">pdf</a>, <a href="https://arxiv.org/format/2101.12634">other</a>] </span>
</p>
<div class="tags is-inline-block">
<span class="tag is-small is-link tooltip is-tooltip-top" data-tooltip="Cosmology and Nongalactic Astrophysics">astro-ph.CO</span>
</div>
</div>
<p class="title is-5 mathjax">
      
        Morphology of relaxed and merging <span class="search-hit mathjax">galaxy</span> clusters. Analytical models for monolithic Minkowski functionals
      
    </p>
<p class="authors">
<span class="has-text-black-bis has-text-weight-semibold">Authors:</span>
<a href="/search/?searchtype=author&amp;query=Schimd%2C+C">C. Schimd</a>, 
      
      <a href="/search/?searchtype=author&amp;query=Sereno%2C+M">M. Sereno</a>
</p>
<p class="abstract mathjax">
<span class="search-hit">Abstract</sp

### Search individual paper by the ID 

In [71]:
search_url = "http://export.arxiv.org/api/query?search_query=id:2101.12634&start=0&max_results=1"

In [73]:
result = urlopen(search_url).read()

In [74]:
xml_result = ET.fromstring(result)

In [75]:
xml_result.

<Element '{http://www.w3.org/2005/Atom}feed' at 0x12ff11f40>

In [76]:
result

b'<?xml version="1.0" encoding="UTF-8"?>\n<feed xmlns="http://www.w3.org/2005/Atom">\n  <link href="http://arxiv.org/api/query?search_query%3Did%3A2101.12634%26id_list%3D%26start%3D0%26max_results%3D1" rel="self" type="application/atom+xml"/>\n  <title type="html">ArXiv Query: search_query=id:2101.12634&amp;id_list=&amp;start=0&amp;max_results=1</title>\n  <id>http://arxiv.org/api/kelp/5/xO5YB6OrWvqogLXbWsjs</id>\n  <updated>2021-02-04T00:00:00-05:00</updated>\n  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:totalResults>\n  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>\n  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:itemsPerPage>\n  <entry>\n    <id>http://arxiv.org/abs/2101.12634v1</id>\n    <updated>2021-01-29T15:15:37Z</updated>\n    <published>2021-01-29T15:15:37Z</published>\n    <title>Morphology of relaxed and merging gala

#### Example:

- [Parsing](https://static.arxiv.org/static/arxiv.marxdown/0.1/help/api/examples/python_arXiv_parsing_example.txt)
- [Paging](https://static.arxiv.org/static/arxiv.marxdown/0.1/help/api/examples/python_arXiv_paging_example.txt)

http://export.arxiv.org/api/query?search_query=cat:astro-ph.GA+AND+ti:galaxy